# 改成螢幕截圖的方式
1. 先把抓截圖的function寫好並執行，功能可以把時間軸與圖給抓下來，並把兩個對應的list存成tuple，方式是用時間判斷，例如每小時回傳一次，那就每小時呼叫一次抓圖片的method，這樣中間才有辦法去做數字的辨識與輸出，不然會卡在一直截圖完全停不下來的窘境
2. 先執行一次，把一部分的圖先抓下來，用trackerbar，透過其中幾張圖把數字的模板給抓出來並存下來給往後使用。
3. 一樣用剛剛存下來的圖，把要辨識的區域座標都給找出來，因為不會變，所以直接用硬編碼的方式寫下來就可以了
4. 把第二步抓下來的數字模板去比對第三步抓下來的數字區域，按照座標排列，並轉成float
5. 然後把資料用dataframe的方式串接起來
6. 輸出成CSV檔存下來

### 先把時間軸與圖給抓下來

In [92]:
## 把一分鐘一次持續一整天的螢幕截圖寫成method，return時間軸與截圖，並創一個screen_shot資料夾把圖片存在當中
def shot(): 
    
    screen_shot = []
    timestamp = []
    n = 0
    localtime_old = time.localtime(time.time())
    localtime_new = localtime_old
    
    ## 00:00:00時回傳出來
    while localtime_new.tm_min == localtime_old.tm_min: ## 這邊要分鐘數一樣才會執行，意思是過到下分鐘就return，結束這個函式
        
        localtime_new = time.localtime(time.time())
        sleep(1) # 幾秒後截圖一次，放在前面才有時間去打開那個畫面
        n += 1
        ## 螢幕截圖
        if n % 30 == 0: ## 在同樣這一分鐘內，每n秒截圖一次
            img = pyautogui.screenshot()
            ## 轉換成opencv讀得懂的ndarray格式
            open_cv_image = np.array(img) 
            # Convert RGB to BGR & copy
            open_cv_image = open_cv_image[:, :, ::-1].copy()

            timestr = time.strftime("%Y%m%d-%H%M%S")
            img_file = "screen_shot/" + timestr +".png"
            cv.imwrite(img_file, open_cv_image)  # 儲存螢幕截圖
            print('Done with output: ' + img_file)

            ## 把截圖跟時間軸存在list中，等等可以直接拿出來做影像處理
            timestampstr = time.strftime("%Y/%m/%d %H:%M:%S")
            timestamp.append(timestampstr)
            screen_shot.append(open_cv_image)
            
            print("n=", n)
        print("new_sec=", localtime_new.tm_sec, "old_min=", localtime_old.tm_min, "new_min=", localtime_new.tm_min)
      
    else:
        ## 回傳一天的螢幕截圖與對應的時間軸
        print("截圖一輪完成")
        return (timestamp, screen_shot)

In [2]:
#看看時間結構長怎樣，要拿出個別時間屬性
 
import time
 
localtime = time.localtime(time.time())
print("本地时间为 :", localtime)

本地时间为 : time.struct_time(tm_year=2023, tm_mon=1, tm_mday=31, tm_hour=18, tm_min=34, tm_sec=58, tm_wday=1, tm_yday=31, tm_isdst=0)


### 定時呼叫一次shot的method來抓資料

In [28]:
## 定時螢幕截圖區塊程式碼，這部分是直接60秒截一次圖並儲存，但要停止只能按上面的停止鈕(KeyboardInterrupt)

import cv2 as cv
import numpy as np

import pyautogui
import sys
import os

from time import sleep
import time

## 創造一個放螢幕截圖的資料夾
try:
    os.mkdir('screen_shot')
except:
    print("The folder existed!")

## 使用shot函式，可以把時間軸與截圖存在list了    
timestamp, screen_shot = shot()

# for t, s in timestamp, screen_shot:
    


The folder existed!
new_sec= 20 old_min= 19 new_min= 19
new_sec= 21 old_min= 19 new_min= 19
new_sec= 22 old_min= 19 new_min= 19
new_sec= 23 old_min= 19 new_min= 19
Done with output: screen_shot/20230131-191925.png
n= 5
new_sec= 24 old_min= 19 new_min= 19
new_sec= 25 old_min= 19 new_min= 19
new_sec= 26 old_min= 19 new_min= 19
new_sec= 27 old_min= 19 new_min= 19
new_sec= 28 old_min= 19 new_min= 19
Done with output: screen_shot/20230131-191930.png
n= 10
new_sec= 29 old_min= 19 new_min= 19
new_sec= 31 old_min= 19 new_min= 19
new_sec= 32 old_min= 19 new_min= 19
new_sec= 33 old_min= 19 new_min= 19
new_sec= 34 old_min= 19 new_min= 19
Done with output: screen_shot/20230131-191936.png
n= 15
new_sec= 35 old_min= 19 new_min= 19
new_sec= 36 old_min= 19 new_min= 19
new_sec= 37 old_min= 19 new_min= 19
new_sec= 38 old_min= 19 new_min= 19
new_sec= 39 old_min= 19 new_min= 19
Done with output: screen_shot/20230131-191941.png
n= 20
new_sec= 40 old_min= 19 new_min= 19
new_sec= 41 old_min= 19 new_min= 19
n

## 這裡往下是先抓一張圖片出來調整位置與閾值

In [76]:
# trackbar調整已截好的螢幕截圖想抓取的區域，目標抓出最適當的辨識圖片與區域
# 使用時注意lower bound不要大於upper bound
import cv2 as cv
import numpy as np

import pyautogui
import sys
import os

from time import sleep
import time

def nothing(x):
    pass

## 螢幕截圖
img = cv.imread('screen_shot/20230131-215832.png')  # 改為之前截好的螢幕截圖
# img = cv.imread('realtime_shot/20220814/screen_shot_1.png')  # 改為之前截好的螢幕截圖
cv.namedWindow('image')

# 因為視窗會隨著圖片大小變動，所以把trackbar與調整的圖片視窗分開
trackbar_window = np.zeros((100,1920,1), np.uint8)
cv.namedWindow('trackbar') 
cv.imshow('trackbar',trackbar_window)

# create trackbars for image size
cv.createTrackbar("lower_Height",'trackbar',0,1080,nothing)  # 調整lower_Height，因為截圖大小的Height為1080
cv.createTrackbar("upper_Height",'trackbar',1,1080,nothing)  # 調整upper_Height，因為截圖大小的Height為1080
cv.createTrackbar('lower_Weight','trackbar',0,1920,nothing)  # 調整lower_Weight，因為截圖大小的Height為1920
cv.createTrackbar('upper_Weight','trackbar',1,1920,nothing)  # 調整upper_Weight，因為截圖大小的Height為1920

# create switch for ON/OFF functionality
switch = '0 : OFF \n1 : ON'
cv.createTrackbar(switch, 'trackbar',0,1,nothing)
res = img   # 第一次要先定義出來


while(1):
    cv.imshow('image',res)

    k = cv.waitKey(1) & 0xFF
    if k == 27:      # Press "Esc" to stop
        break
    # get current positions of four trackbars
    lower_Height = cv.getTrackbarPos("lower_Height",'trackbar')
    upper_Height = cv.getTrackbarPos("upper_Height",'trackbar')
    lower_Weight = cv.getTrackbarPos("lower_Weight",'trackbar')
    upper_Weight = cv.getTrackbarPos("upper_Weight",'trackbar')

    s = cv.getTrackbarPos(switch,'trackbar')
    if s == 0:
        res[:] = img
    else:
        
        res = img[lower_Height:upper_Height, lower_Weight:upper_Weight]

cv.destroyAllWindows()

### 下面這格不要再執行了，不然會把抓下來的數字模板都給覆蓋掉

In [ ]:
# ## 這格不用再次執行，如果資料夾中有數字的模板圖的話
# ## 這格用來把數字擷取下來先存下來，但是是從多張截圖來的，因為一張截圖不一定有全部數字

# ### 各數字圖片範圍紀錄
# zero = img[302:318, 462:477]
# one =  img[324:340, 412:423]
# two =  img[324:340, 440:455]
# three =  img[324:340, 427:440]
# four = img[302:318, 396:412]
# five = img[337:352, 1130:1144]
# six = img[312:327, 1141:1155]
# seven = img[312:327, 1125:1141]
# eight = img[337:353, 1144:1159]
# nine = img[282:296, 382:397]
# dot = img[289:296, 397:405]
# minus = img[337:353, 1098:1108]

# numbers = [zero, one, two, three, four, five, six, seven, eight, nine, dot, minus]

# for number in range(len(numbers)):
    
    
#     number_str = str(number)
#     template_file = "screen_shot/" + number_str +".png"
#     cv.imwrite(template_file, numbers[number])  # 儲存螢幕截圖
#     print('Done with output: ' + template_file)

### 下面這格要再執行，因為數字模板抓下來，之後要用得抓進來用

In [5]:
### 把剛抓下來的數字模板抓進來

number_template = []
for i in range(12):
    template_str = "screen_shot/" + str(i) + ".png"
    template = cv.imread(template_str)  # 改為之前截好的螢幕截圖
    number_template.append(template)

In [85]:
## 把欲辨識區域範圍挖出來
CPC_flowA = img[281:298, 366:479]
CPC_flowB = img[302:319, 366:479]
CPC_pressure = img[322:343, 375:470]
USI_flow = img[288:308, 1081:1194]
USI_pressure = img[309:330, 1090:1185]
bias_flow = img[337:353, 1087:1188]

# 把要辨識的區塊全部截取出來放在number_Tag中
number_Tag = {"CPC_flowA":CPC_flowA , "CPC_flowB":CPC_flowB, "CPC_pressure":CPC_pressure, "USI_flow":USI_flow, 
              "USI_pressure":USI_pressure, "bias_flow":bias_flow}

In [86]:
# 這格印出來看看而已，把剛剛抓出來的數字模板，放入roi中，看看印出來是不是正確
roi = bias_flow # 截圖大小: (1080, 1920, 3)
cv.imshow("Display window", roi)
k = cv.waitKey(0)     # Wait for a keystroke in the window (e.g.,"a")
cv.destroyAllWindows()

In [10]:
# 用cv.findContours()來比對圖片
# 迴圈抓每張圖片，然後迴圈每個數字比較一次，再用字典的方式放進去
# 然後用座標的位置排序，按照順序把數字、小數點、正負號組成字串，再轉成float
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

all_target = {}
for target, region in number_Tag.items():
    tag = region 
    img_rgb = tag.copy()
    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    coordinate = {}
    for index, number in enumerate(number_template):
        template = cv.cvtColor(number, cv.COLOR_BGR2GRAY) # 一個一個比對，把圖都轉成灰階再來比對

        w, h = template.shape[::-1]
        res = cv.matchTemplate(img_gray,template,cv.TM_CCOEFF_NORMED) # 用cv.TM_CCOEFF_NORMED這個演算法做
        
        ## 測試發現0這個數字要比較低的閾值，可能0中間那個洞的型態長不大一樣八
        if index == 0:
            threshold = 0.85
        elif index == 10:
            threshold = 0.9
        elif index == 11:
            threshold = 0.7
        else:
            threshold = 0.87
        loc = np.where( res >= threshold)
        

        coordinate[index] = loc[1]
        
    ## 將抓到的數字座標位置轉換為數字
    number = []
    number_map = {}

    for i,n in coordinate.items():
        for num in n:
            number_map[num] = i

    for index, value in enumerate(sorted(number_map)):
        if number_map[value] == 10:
            number.append(".")
        elif number_map[value] == 11:
            number.append("-")
        else:
            number.append(str(number_map[value]))

    number_str = "".join(number)
#     print(number_str) # 一開始沒調好時，要先把字串印出來，調好了再轉float
    number_float = float(number_str)
        
    all_target[target] = number_float
print(all_target)
    

{'CPC_flowA': 9.0, 'CPC_flowB': 7695.0, 'CPC_pressure': 27.056, 'USI_flow': 7700.0, 'USI_pressure': 26.7, 'bias_flow': 0.052}


### 上面是先拿一張圖做，這裡往下改為實時截圖下來的東西來做

In [25]:
### 把剛抓下來的數字模板抓進來

number_template = []
for i in range(12):
    template_str = "screen_shot/" + str(i) + ".png"
    template = cv.imread(template_str)  # 改為之前截好的螢幕截圖
    number_template.append(template)

### 這格會開始截圖抓資料，然後用比對時間的方式判斷什麼時候要結束，目前預設是1分鐘一筆，00:00結束，一天的資料放在一個CSV中，檔名用當天的日期命名

In [96]:
## 定時螢幕截圖區塊程式碼，這部分是直接X秒(在shot()裡設定)截一次圖並儲存，要停止可以按上面的停止鈕(KeyboardInterrupt)，不然就是等到時間比對到

import cv2 as cv
import numpy as np

import pyautogui
import sys
import os

from time import sleep
import time

import pandas as pd
from matplotlib import pyplot as plt

## 創造一個放螢幕截圖與CSV檔的資料夾
try:
    os.mkdir('screen_shot')
except:
    print("The screen_shot folder existed!")
try:
    os.mkdir('CSV_data')
except:
    print("The CSV_data folder existed!")

n = 0
localtime_old = time.localtime(time.time())
localtime_new = localtime_old

## 一次就可以執行一天，預計把一整天的資料放入一個CSV中
while localtime_new.tm_mday == localtime_old.tm_mday: 
    localtime_new = time.localtime(time.time())

    ## 使用shot函式，可以把時間軸與截圖存在list了    
    timestamp, screen_shot = shot()

    # for t, s in timestamp, screen_shot:
    
    
    one_round_data = []

    for one_shot in screen_shot:

        # 把每個欲辨識的區域都先挖出來，並放入一個dict中
        CPC_flowA = one_shot[281:298, 366:479]
        CPC_flowB = one_shot[302:319, 366:479]
        CPC_pressure = one_shot[322:343, 375:470]
        USI_flow = one_shot[288:308, 1081:1194]
        USI_pressure = one_shot[309:330, 1090:1185]
        bias_flow = one_shot[337:353, 1087:1188]
        number_Tag = {"CPC_flowA":CPC_flowA , "CPC_flowB":CPC_flowB, "CPC_pressure":CPC_pressure, "USI_flow":USI_flow, 
                  "USI_pressure":USI_pressure, "bias_flow":bias_flow}

        # 每跑一次迴圈就辨識一次圖，把數字抓出來
        all_target = {}
        for target, region in number_Tag.items():
            tag = region 
            img_rgb = tag.copy()
            img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
            coordinate = {}
            for index, number in enumerate(number_template):
                template = cv.cvtColor(number, cv.COLOR_BGR2GRAY) # 一個一個比對

                w, h = template.shape[::-1]
                res = cv.matchTemplate(img_gray,template,cv.TM_CCOEFF_NORMED) # 用cv.TM_CCOEFF_NORMED這個演算法做

                ## 測試發現0這個數字要比較低的閾值，可能0中間那個洞的型態長不大一樣八
                if index == 0:
                    threshold = 0.85
                elif index == 10:
                    threshold = 0.89
                elif index == 11:
                    threshold = 0.7
                else:
                    threshold = 0.87
                loc = np.where( res >= threshold)


                coordinate[index] = loc[1]

            ## 將抓到的數字座標位置轉換為數字
            number = []
            number_map = {}

            for i,n in coordinate.items():
                for num in n:
                    number_map[num] = i

            for index, value in enumerate(sorted(number_map)):
                if number_map[value] == 10:
                    number.append(".")
                elif number_map[value] == 11:
                    number.append("-")
                else:
                    number.append(str(number_map[value]))

            number_str = "".join(number)
    #         print(number_str) # 一開始沒調好時，要先把字串印出來，調好了再轉float
            number_float = float(number_str)

            all_target[target] = number_float

        one_round_data.append(all_target)
    # print(len(one_round_data))

    df = pd.DataFrame(one_round_data, index=timestamp)
    
    if n == 0:
        all_df = df
    else:
        all_df = pd.concat([all_df, df])

    # 把資料輸出成CSV檔
    mydir = './CSV_data/'
    csv_file = os.path.join(mydir, f"{time.localtime(time.time()).tm_year}{time.localtime(time.time()).tm_mon:02d}{time.localtime(time.time()).tm_mday}.csv")
    all_df.to_csv(csv_file, index=True)
    
    n = n + 1

else:
    print(localtime_old.tm_mday, "日的資料輸出結束")


The folder existed!
The folder existed!
new_sec= 9 old_min= 29 new_min= 29
new_sec= 10 old_min= 29 new_min= 29
new_sec= 11 old_min= 29 new_min= 29
new_sec= 12 old_min= 29 new_min= 29
new_sec= 13 old_min= 29 new_min= 29
new_sec= 14 old_min= 29 new_min= 29
new_sec= 15 old_min= 29 new_min= 29
new_sec= 16 old_min= 29 new_min= 29
new_sec= 17 old_min= 29 new_min= 29
new_sec= 18 old_min= 29 new_min= 29
new_sec= 19 old_min= 29 new_min= 29
new_sec= 20 old_min= 29 new_min= 29
new_sec= 22 old_min= 29 new_min= 29
new_sec= 23 old_min= 29 new_min= 29
new_sec= 24 old_min= 29 new_min= 29
new_sec= 25 old_min= 29 new_min= 29
new_sec= 26 old_min= 29 new_min= 29
new_sec= 27 old_min= 29 new_min= 29
new_sec= 28 old_min= 29 new_min= 29
new_sec= 29 old_min= 29 new_min= 29
new_sec= 30 old_min= 29 new_min= 29
new_sec= 31 old_min= 29 new_min= 29
new_sec= 32 old_min= 29 new_min= 29
new_sec= 33 old_min= 29 new_min= 29
new_sec= 34 old_min= 29 new_min= 29
new_sec= 35 old_min= 29 new_min= 29
new_sec= 36 old_min= 29 n

C:\TEMP/ipykernel_13568/3601228380.py:108: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if n == 0:


new_sec= 1 old_min= 30 new_min= 30
new_sec= 2 old_min= 30 new_min= 30
new_sec= 3 old_min= 30 new_min= 30
new_sec= 4 old_min= 30 new_min= 30
new_sec= 5 old_min= 30 new_min= 30
new_sec= 6 old_min= 30 new_min= 30
new_sec= 7 old_min= 30 new_min= 30
new_sec= 8 old_min= 30 new_min= 30
new_sec= 9 old_min= 30 new_min= 30
new_sec= 10 old_min= 30 new_min= 30
new_sec= 11 old_min= 30 new_min= 30
new_sec= 12 old_min= 30 new_min= 30
new_sec= 13 old_min= 30 new_min= 30
new_sec= 14 old_min= 30 new_min= 30
new_sec= 15 old_min= 30 new_min= 30
new_sec= 16 old_min= 30 new_min= 30
new_sec= 17 old_min= 30 new_min= 30
new_sec= 18 old_min= 30 new_min= 30
new_sec= 19 old_min= 30 new_min= 30
new_sec= 20 old_min= 30 new_min= 30
new_sec= 21 old_min= 30 new_min= 30
new_sec= 22 old_min= 30 new_min= 30
new_sec= 23 old_min= 30 new_min= 30
new_sec= 24 old_min= 30 new_min= 30
new_sec= 25 old_min= 30 new_min= 30
new_sec= 26 old_min= 30 new_min= 30
new_sec= 27 old_min= 30 new_min= 30
new_sec= 28 old_min= 30 new_min= 30
n

new_sec= 42 old_min= 33 new_min= 33
new_sec= 43 old_min= 33 new_min= 33
new_sec= 44 old_min= 33 new_min= 33
new_sec= 45 old_min= 33 new_min= 33
new_sec= 46 old_min= 33 new_min= 33
new_sec= 47 old_min= 33 new_min= 33
new_sec= 48 old_min= 33 new_min= 33
new_sec= 49 old_min= 33 new_min= 33
new_sec= 50 old_min= 33 new_min= 33
new_sec= 51 old_min= 33 new_min= 33
new_sec= 52 old_min= 33 new_min= 33
new_sec= 53 old_min= 33 new_min= 33
new_sec= 54 old_min= 33 new_min= 33
new_sec= 55 old_min= 33 new_min= 33
new_sec= 56 old_min= 33 new_min= 33
new_sec= 57 old_min= 33 new_min= 33
new_sec= 58 old_min= 33 new_min= 33
new_sec= 59 old_min= 33 new_min= 33
new_sec= 0 old_min= 33 new_min= 34
截圖一輪完成
new_sec= 1 old_min= 34 new_min= 34
new_sec= 2 old_min= 34 new_min= 34
new_sec= 3 old_min= 34 new_min= 34
new_sec= 4 old_min= 34 new_min= 34
new_sec= 5 old_min= 34 new_min= 34
new_sec= 6 old_min= 34 new_min= 34
new_sec= 7 old_min= 34 new_min= 34
new_sec= 8 old_min= 34 new_min= 34
new_sec= 9 old_min= 34 new_min

new_sec= 26 old_min= 37 new_min= 37
new_sec= 27 old_min= 37 new_min= 37
new_sec= 28 old_min= 37 new_min= 37
new_sec= 29 old_min= 37 new_min= 37
Done with output: screen_shot/20230131-223731.png
n= 30
new_sec= 30 old_min= 37 new_min= 37
new_sec= 31 old_min= 37 new_min= 37
new_sec= 32 old_min= 37 new_min= 37
new_sec= 33 old_min= 37 new_min= 37
new_sec= 34 old_min= 37 new_min= 37
new_sec= 35 old_min= 37 new_min= 37
new_sec= 36 old_min= 37 new_min= 37
new_sec= 37 old_min= 37 new_min= 37
new_sec= 38 old_min= 37 new_min= 37
new_sec= 39 old_min= 37 new_min= 37
new_sec= 40 old_min= 37 new_min= 37
new_sec= 41 old_min= 37 new_min= 37
new_sec= 42 old_min= 37 new_min= 37
new_sec= 43 old_min= 37 new_min= 37
new_sec= 44 old_min= 37 new_min= 37
new_sec= 45 old_min= 37 new_min= 37
new_sec= 46 old_min= 37 new_min= 37
new_sec= 47 old_min= 37 new_min= 37
new_sec= 48 old_min= 37 new_min= 37
new_sec= 49 old_min= 37 new_min= 37
new_sec= 50 old_min= 37 new_min= 37
new_sec= 51 old_min= 37 new_min= 37
new_sec=

...

new_sec= 14 old_min= 48 new_min= 48
new_sec= 15 old_min= 48 new_min= 48
new_sec= 16 old_min= 48 new_min= 48
new_sec= 17 old_min= 48 new_min= 48
new_sec= 18 old_min= 48 new_min= 48
new_sec= 19 old_min= 48 new_min= 48
new_sec= 20 old_min= 48 new_min= 48
new_sec= 21 old_min= 48 new_min= 48
new_sec= 22 old_min= 48 new_min= 48
new_sec= 23 old_min= 48 new_min= 48
new_sec= 24 old_min= 48 new_min= 48
new_sec= 25 old_min= 48 new_min= 48
new_sec= 26 old_min= 48 new_min= 48
new_sec= 27 old_min= 48 new_min= 48
new_sec= 28 old_min= 48 new_min= 48
new_sec= 29 old_min= 48 new_min= 48
Done with output: screen_shot/20230131-234832.png
n= 30
new_sec= 30 old_min= 48 new_min= 48
new_sec= 32 old_min= 48 new_min= 48
new_sec= 33 old_min= 48 new_min= 48
new_sec= 34 old_min= 48 new_min= 48
new_sec= 35 old_min= 48 new_min= 48
new_sec= 36 old_min= 48 new_min= 48
new_sec= 37 old_min= 48 new_min= 48
new_sec= 38 old_min= 48 new_min= 48
new_sec= 39 old_min= 48 new_min= 48
new_sec= 40 old_min= 48 new_min= 48
new_sec=

new_sec= 57 old_min= 51 new_min= 51
new_sec= 58 old_min= 51 new_min= 51
new_sec= 59 old_min= 51 new_min= 51
Done with output: screen_shot/20230131-235201.png
n= 60
new_sec= 0 old_min= 51 new_min= 52
截圖一輪完成
new_sec= 1 old_min= 52 new_min= 52
new_sec= 2 old_min= 52 new_min= 52
new_sec= 3 old_min= 52 new_min= 52
new_sec= 4 old_min= 52 new_min= 52
new_sec= 5 old_min= 52 new_min= 52
new_sec= 6 old_min= 52 new_min= 52
new_sec= 7 old_min= 52 new_min= 52
new_sec= 8 old_min= 52 new_min= 52
new_sec= 9 old_min= 52 new_min= 52
new_sec= 10 old_min= 52 new_min= 52
new_sec= 11 old_min= 52 new_min= 52
new_sec= 12 old_min= 52 new_min= 52
new_sec= 13 old_min= 52 new_min= 52
new_sec= 14 old_min= 52 new_min= 52
new_sec= 15 old_min= 52 new_min= 52
new_sec= 16 old_min= 52 new_min= 52
new_sec= 17 old_min= 52 new_min= 52
new_sec= 18 old_min= 52 new_min= 52
new_sec= 19 old_min= 52 new_min= 52
new_sec= 20 old_min= 52 new_min= 52
new_sec= 21 old_min= 52 new_min= 52
new_sec= 22 old_min= 52 new_min= 52
new_sec= 23

new_sec= 38 old_min= 55 new_min= 55
new_sec= 39 old_min= 55 new_min= 55
new_sec= 40 old_min= 55 new_min= 55
new_sec= 41 old_min= 55 new_min= 55
new_sec= 42 old_min= 55 new_min= 55
new_sec= 43 old_min= 55 new_min= 55
new_sec= 44 old_min= 55 new_min= 55
new_sec= 45 old_min= 55 new_min= 55
new_sec= 46 old_min= 55 new_min= 55
new_sec= 47 old_min= 55 new_min= 55
new_sec= 48 old_min= 55 new_min= 55
new_sec= 49 old_min= 55 new_min= 55
new_sec= 50 old_min= 55 new_min= 55
new_sec= 51 old_min= 55 new_min= 55
new_sec= 52 old_min= 55 new_min= 55
new_sec= 53 old_min= 55 new_min= 55
new_sec= 54 old_min= 55 new_min= 55
new_sec= 55 old_min= 55 new_min= 55
new_sec= 56 old_min= 55 new_min= 55
new_sec= 57 old_min= 55 new_min= 55
new_sec= 58 old_min= 55 new_min= 55
new_sec= 59 old_min= 55 new_min= 55
Done with output: screen_shot/20230131-235601.png
n= 60
new_sec= 0 old_min= 55 new_min= 56
截圖一輪完成
new_sec= 1 old_min= 56 new_min= 56
new_sec= 2 old_min= 56 new_min= 56
new_sec= 3 old_min= 56 new_min= 56
new_s

new_sec= 21 old_min= 59 new_min= 59
new_sec= 22 old_min= 59 new_min= 59
new_sec= 23 old_min= 59 new_min= 59
new_sec= 24 old_min= 59 new_min= 59
new_sec= 25 old_min= 59 new_min= 59
new_sec= 26 old_min= 59 new_min= 59
new_sec= 27 old_min= 59 new_min= 59
new_sec= 28 old_min= 59 new_min= 59
new_sec= 29 old_min= 59 new_min= 59
Done with output: screen_shot/20230131-235931.png
n= 30
new_sec= 30 old_min= 59 new_min= 59
new_sec= 31 old_min= 59 new_min= 59
new_sec= 32 old_min= 59 new_min= 59
new_sec= 33 old_min= 59 new_min= 59
new_sec= 34 old_min= 59 new_min= 59
new_sec= 35 old_min= 59 new_min= 59
new_sec= 36 old_min= 59 new_min= 59
new_sec= 37 old_min= 59 new_min= 59
new_sec= 38 old_min= 59 new_min= 59
new_sec= 39 old_min= 59 new_min= 59
new_sec= 40 old_min= 59 new_min= 59
new_sec= 41 old_min= 59 new_min= 59
new_sec= 42 old_min= 59 new_min= 59
new_sec= 43 old_min= 59 new_min= 59
new_sec= 44 old_min= 59 new_min= 59
new_sec= 45 old_min= 59 new_min= 59
new_sec= 46 old_min= 59 new_min= 59
new_sec=

AttributeError: 'time.struct_time' object has no attribute 'mday'

### 這格式用來先試抓一個shot看看用的，不執行沒關係

In [39]:
import cv2 as cv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tempfile, os.path

one_round_data = []

for one_shot in screen_shot:

    # 把每個欲辨識的區域都先挖出來，並放入一個dict中
    CPC_flowA = one_shot[281:298, 380:464]
    CPC_flowB = one_shot[302:319, 366:479]
    CPC_pressure = one_shot[322:343, 375:470]
    USI_flow = one_shot[288:308, 1081:1194]
    USI_pressure = one_shot[309:330, 1090:1185]
    bias_flow = one_shot[337:353, 1099:1175]
    number_Tag = {"CPC_flowA":CPC_flowA , "CPC_flowB":CPC_flowB, "CPC_pressure":CPC_pressure, "USI_flow":USI_flow, 
              "USI_pressure":USI_pressure, "bias_flow":bias_flow}
    
    # 每跑一次迴圈就辨識一次圖，把數字抓出來
    all_target = {}
    for target, region in number_Tag.items():
        tag = region 
        img_rgb = tag.copy()
        img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
        coordinate = {}
        for index, number in enumerate(number_template):
            template = cv.cvtColor(number, cv.COLOR_BGR2GRAY) # 一個一個比對

            w, h = template.shape[::-1]
            res = cv.matchTemplate(img_gray,template,cv.TM_CCOEFF_NORMED) # 用cv.TM_CCOEFF_NORMED這個演算法做

            ## 測試發現0這個數字要比較低的閾值，可能0中間那個洞的型態長不大一樣八
            if index == 0:
                threshold = 0.85
            elif index == 10:
                threshold = 0.89
            elif index == 11:
                threshold = 0.7
            else:
                threshold = 0.87
            loc = np.where( res >= threshold)


            coordinate[index] = loc[1]

        ## 將抓到的數字座標位置轉換為數字
        number = []
        number_map = {}

        for i,n in coordinate.items():
            for num in n:
                number_map[num] = i

        for index, value in enumerate(sorted(number_map)):
            if number_map[value] == 10:
                number.append(".")
            elif number_map[value] == 11:
                number.append("-")
            else:
                number.append(str(number_map[value]))

        number_str = "".join(number)
#         print(number_str) # 一開始沒調好時，要先把字串印出來，調好了再轉float
        number_float = float(number_str)

        all_target[target] = number_float
        
    one_round_data.append(all_target)
# print(len(one_round_data))

df = pd.DataFrame(one_round_data, index=timestamp)

if 

# 把資料輸出成CSV檔
mydir = './CSV_data/'
csv_file = os.path.join(mydir, "users.csv")
df.to_csv(csv_file, index=True)


7


In [40]:
print(len(timestamp))
print(df)

7
                     CPC_flowA  CPC_flowB  CPC_pressure  USI_flow  \
2023/01/31 19:19:25        4.0        0.0        25.044       0.0   
2023/01/31 19:19:31        4.0        0.0        25.044       0.0   
2023/01/31 19:19:36        4.0        0.0        25.044       0.0   
2023/01/31 19:19:41        4.0        0.0        25.044       0.0   
2023/01/31 19:19:46        4.0        0.0        25.044       0.0   
2023/01/31 19:19:51        4.0        0.0        25.044       0.0   
2023/01/31 19:19:57        4.0        0.0        25.044       0.0   

                     USI_pressure  bias_flow  
2023/01/31 19:19:25         25.48        0.0  
2023/01/31 19:19:31         25.48        0.0  
2023/01/31 19:19:36         25.48        0.0  
2023/01/31 19:19:41         25.48        0.0  
2023/01/31 19:19:46         25.48        0.0  
2023/01/31 19:19:51         25.48        0.0  
2023/01/31 19:19:57         25.48        0.0  


In [45]:
df_all = df

In [46]:
df_all = pd.concat([df_all, df])

In [47]:
df_all # dataframe串接起來後看看長怎樣

,CPC_flowA,CPC_flowB,CPC_pressure,USI_flow,USI_pressure,bias_flow
2023/01/31 19:19:25,4.0,0.0,25.044,0.0,25.48,0.0
2023/01/31 19:19:31,4.0,0.0,25.044,0.0,25.48,0.0
2023/01/31 19:19:36,4.0,0.0,25.044,0.0,25.48,0.0
2023/01/31 19:19:41,4.0,0.0,25.044,0.0,25.48,0.0
2023/01/31 19:19:46,4.0,0.0,25.044,0.0,25.48,0.0
2023/01/31 19:19:51,4.0,0.0,25.044,0.0,25.48,0.0
2023/01/31 19:19:57,4.0,0.0,25.044,0.0,25.48,0.0
2023/01/31 19:19:25,4.0,0.0,25.044,0.0,25.48,0.0
2023/01/31 19:19:31,4.0,0.0,25.044,0.0,25.48,0.0
2023/01/31 19:19:36,4.0,0.0,25.044,0.0,25.48,0.0
